## Rotina para cálculo da energia potencial total utilizando computação simbólica
### Viga de Euler-Bernoulli bi-apoiada

In [28]:
# Importar todos os comandos da biblioteca sympy
from sympy import *
# Para uma impressão mais elegante:
init_printing(use_unicode=True)
# Declarar variáveis simbólicas
x = Symbol('x')
E = Symbol('E')
I = Symbol('I')
L,q = symbols('L q')

### Cálculo de $\Pi = U + W$

In [32]:
I = 0.2*(0.6**3)/12
print(I)
E = 200e9
print(E*I)

0.0035999999999999995
719999999.9999999


In [3]:

# === Parâmetros físicos e constantes ===
EI = 7.2e8   # Rigidez à flexão [N·m²]
cf = 4e8     # Coeficiente de mola elástica (fundo de Winkler) [N/m²]
q0 = 5e3     # Carga distribuída [N/m]
F0 = 2e3     # Carga concentrada na extremidade [N]
M0 = 4e3     # Momento concentrado na extremidade [N·m]
L_val = 8    # Comprimento do vão [m]

# === Definição simbólica das funções ===
v = Function('v')(x)     # Função de deslocamento
w = Function('w')(x)     # Função de teste

# === Derivadas de ordem 2 ===
v_xx = diff(v, x, 2)
w_xx = diff(w, x, 2)

# === Forma bilinear ===
# a(v, w) = ∫₀ᴸ [EI·v''·w'' + cf·v·w] dx
bilinear_form = integrate(EI * v_xx * w_xx + cf * v * w, (x, 0, L))

# === Forma linear ===
# l(w) = ∫₀ᴸ q₀·w dx + F₀·w(L) - M₀·w'(L)
w_prime = diff(w, x)
linear_form = integrate(q0 * w, (x, 0, L)) + F0 * w.subs(x, L) - M0 * w_prime.subs(x, L)

# === Impressão dos resultados ===
print("Forma bilinear a(v, w):")
pprint(bilinear_form, use_unicode=True)

print("\nForma linear l(w):")
pprint(linear_form, use_unicode=True)

Forma bilinear a(v, w):
            ⎛                                           L                      ↪
            ⎜                                           ⌠                      ↪
            ⎜    L                                      ⎮      2         2     ↪
            ⎜    ⌠                                      ⎮     d         d      ↪
720000000.0⋅⎜1.0⋅⎮ 0.555555555555556⋅v(x)⋅w(x) dx + 1.0⋅⎮ 1.0⋅───(v(x))⋅───(w( ↪
            ⎜    ⌡                                      ⎮       2         2    ↪
            ⎜    0                                      ⎮     dx        dx     ↪
            ⎜                                           ⌡                      ↪
            ⎝                                           0                      ↪

↪       ⎞
↪       ⎟
↪       ⎟
↪       ⎟
↪ x)) dx⎟
↪       ⎟
↪       ⎟
↪       ⎟
↪       ⎠

Forma linear l(w):
                                       L        
                     d                 ⌠        
2000.0⋅w(L) - 4000.0⋅──(w(L)) + 5000.0

### Impressão do resultado numérico

In [6]:
# Define symbolic displacement function
v = Function('v')(x)

# Second derivative
v_xx = diff(v, x, 2)

# Define the functional: Π(v) = 1/2 ∫(EI (v'')^2 + cf v^2) dx - ∫q0 v dx - F0 v(L) + M0 v'(L)
strain_energy = (1/2) * integrate(EI * v_xx**2 + cf * v**2, (x, 0, L))
external_work = integrate(q0 * v, (x, 0, L)) + F0 * v.subs(x, L) - M0 * diff(v, x).subs(x, L)
Pi = strain_energy - external_work

# Print the functional
print("Total Potential Energy Functional Π(v):")
pprint(Pi, use_unicode = True)

Total Potential Energy Functional Π(v):
                                                                               ↪
                                                                               ↪
                                                                L              ↪
                                        L                       ⌠              ↪
                      d                 ⌠                       ⎮              ↪
-2000.0⋅v(L) + 4000.0⋅──(v(L)) - 5000.0⋅⎮ v(x) dx + 360000000.0⋅⎮ 0.5555555555 ↪
                      dL                ⌡                       ⌡              ↪
                                        0                       0              ↪
                                                                               ↪
                                                                               ↪

↪                              L                    
↪                              ⌠                    
↪                              ⎮           

In [11]:
F = (1/2) * EI * v_xx**2 + (1/2) * cf * v**2 - q0 * v

# Compute partial derivatives for Euler-Lagrange equation
partial_F_v = diff(F, v)
partial_F_v_prime = diff(F, diff(v, x))
partial_F_v_double_prime = diff(F, v_xx)

# Compute the Euler-Lagrange equation
euler_lagrange = partial_F_v - diff(partial_F_v_prime, x) + diff(partial_F_v_double_prime, x, 2)

# Print the Euler-Lagrange equation
print("Euler-Lagrange Equation:")
pprint(euler_lagrange, use_unicode = True)

# Boundary terms
v_prime = diff(v, x)
v_xxx = diff(v, x, 3)
boundary_term_v = -diff(partial_F_v_double_prime, x) - F0  # At x = L
boundary_term_v_prime = partial_F_v_double_prime - M0  # At x = L

print("\nBoundary condition at x = L (for v):")
print(boundary_term_v.subs(x, L), "= 0")
print("Boundary condition at x = L (for v'):")
print(boundary_term_v_prime.subs(x, L), "= 0")

Euler-Lagrange Equation:
                                4                
                               d                 
400000000.0⋅v(x) + 720000000.0⋅───(v(x)) - 5000.0
                                 4               
                               dx                

Boundary condition at x = L (for v):
-720000000.0*Derivative(v(L), (L, 3)) - 2000.0 = 0
Boundary condition at x = L (for v'):
720000000.0*Derivative(v(L), (L, 2)) - 4000.0 = 0


In [12]:
# Define basis functions
phi_1 = x**2
phi_2 = x**3

# Compute second derivatives
phi_1_xx = phi_1.diff(x, 2)  # 2
phi_2_xx = phi_2.diff(x, 2)  # 6*x

# Compute K_ij
K_11 = integrate(EI * phi_1_xx * phi_1_xx + cf * phi_1 * phi_1, (x, 0, L))
K_12 = integrate(EI * phi_1_xx * phi_2_xx + cf * phi_1 * phi_2, (x, 0, L))
K_21 = K_12  # Symmetry
K_22 = integrate(EI * phi_2_xx * phi_2_xx + cf * phi_2 * phi_2, (x, 0, L))

# Compute F_i
F_1 = integrate(q0 * phi_1, (x, 0, L)) + F0 * phi_1.subs(x, L) - M0 * phi_1.diff(x).subs(x, L)
F_2 = integrate(q0 * phi_2, (x, 0, L)) + F0 * phi_2.subs(x, L) - M0 * phi_2.diff(x).subs(x, L)

# Print results
print("Stiffness Matrix K_ij:")
print(f"K_11 = {K_11}")
print(f"K_12 = K_21 = {K_12}")
print(f"K_22 = {K_22}")
print("\nForce Vector F_i:")
print(f"F_1 = {F_1}")
print(f"F_2 = {F_2}")

Stiffness Matrix K_ij:
K_11 = 80000000.0*L**5 + 2880000000.0*L
K_12 = K_21 = 66666666.6666667*L**6 + 4320000000.0*L**2
K_22 = 57142857.1428571*L**7 + 8640000000.0*L**3

Force Vector F_i:
F_1 = 1666.66666666667*L**3 + 2000.0*L**2 - 8000.0*L
F_2 = 1250.0*L**4 + 2000.0*L**3 - 12000.0*L**2


In [13]:
# Define basis functions up to 6th order
phi = [x**(i+1) for i in range(1, 6)]  # x^2, x^3, x^4, x^5, x^6

# Function to compute K_ij and F_i for a given n
def compute_system(n):
    K = Matrix.zeros(n, n)
    F = Matrix.zeros(n, 1)
    
    for i in range(n):
        for j in range(n):
            phi_i_xx = phi[i].diff(x, 2)
            phi_j_xx = phi[j].diff(x, 2)
            K[i, j] = integrate(EI * phi_i_xx * phi_j_xx + cf * phi[i] * phi[j], (x, 0, L))
        
        F[i] = integrate(q0 * phi[i], (x, 0, L)) + F0 * phi[i].subs(x, L) - M0 * phi[i].diff(x).subs(x, L)
    
    return K, F

# Compute and solve for each order
solutions = {}
for n in range(1, 6):
    K, F = compute_system(n)
    c = Matrix(symbols(f'c1:{n+1}'))
    solution = solve(K * c - F, c)
    
    # Construct approximate solution
    v_approx = sum(solution.get(symbols(f'c{i+1}'), 0) * phi[i] for i in range(n))
    solutions[n] = (K, F, solution, v_approx)
    
    # Print results
    print(f"\nApproximation order {n} (degree {n+1}):")
    print("Stiffness Matrix K:")
    print(K)
    print("Force Vector F:")
    print(F)
    print("Coefficients:")
    print(solution)
    print("Approximate solution v(x):")
    print(v_approx)

# Numerical evaluation with L = 8
print("\nNumerical results with L = 8:")
for n in range(1, 6):
    K, F, solution, v_approx = solutions[n]
    K_num = K.subs(L, L_val)
    F_num = F.subs(L, L_val)
    solution_num = {k: v.subs(L, L_val).evalf() for k, v in solution.items()}
    v_approx_num = v_approx.subs(L, L_val).subs(solution_num)
    
    print(f"\nOrder {n} (degree {n+1}):")
    print("Stiffness Matrix K (numerical):")
    print(K_num.evalf())
    print("Force Vector F (numerical):")
    print(F_num.evalf())
    print("Coefficients (numerical):")
    print(solution_num)
    print("Approximate solution v(x):")
    print(v_approx_num)


Approximation order 1 (degree 2):
Stiffness Matrix K:
Matrix([[80000000.0*L**5 + 2880000000.0*L]])
Force Vector F:
Matrix([[1666.66666666667*L**3 + 2000.0*L**2 - 8000.0*L]])
Coefficients:
{c1: (5.0*L**2 + 6.0*L - 24.0)/(240000.0*L**4 + 8640000.0)}
Approximate solution v(x):
x**2*(5.0*L**2 + 6.0*L - 24.0)/(240000.0*L**4 + 8640000.0)

Approximation order 2 (degree 3):
Stiffness Matrix K:
Matrix([[80000000.0*L**5 + 2880000000.0*L, 66666666.6666667*L**6 + 4320000000.0*L**2], [66666666.6666667*L**6 + 4320000000.0*L**2, 57142857.1428571*L**7 + 8640000000.0*L**3]])
Force Vector F:
Matrix([[1666.66666666667*L**3 + 2000.0*L**2 - 8000.0*L], [1250.0*L**4 + 2000.0*L**3 - 12000.0*L**2]])
Coefficients:
{c1: (3.57142857142854e+24*L**6 - 5.71428571428576e+24*L**5 + 1.02857142857143e+26*L**4 + 2.7e+27*L**2 + 2.592e+27*L - 5.184e+27)/(3.80952380952357e+28*L**8 + 8.39314285714285e+31*L**4 + 1.86624e+33), c2: (-3.33333333333335e+24*L**5 + 7.99999999999998e+24*L**4 - 1.28e+26*L**3 - 1.08e+27*L - 8.64e+26)

In [40]:
from sympy import symbols, integrate, Matrix, solve, diff, cos, sin, exp
import numpy as np
import matplotlib.pyplot as plt


# Define symbolic variables
x, L = symbols('x L')
c1, c2, c3, c4 = symbols('c1 c2 c3 c4')

# Basis functions
phi = [x**(i+1) for i in range(1, 6)]  # x^2, x^3, x^4, x^5, x^6

beta = (cf / (4 * EI)) ** 0.25

print(beta)

# Compute K_ij and F_i
def compute_system(n):
    #print(n)
    K = Matrix.zeros(n, n)
    F = Matrix.zeros(n, 1)
    for i in range(n):
        for j in range(n):
            phi_i_xx = phi[i].diff(x, 2)
            phi_j_xx = phi[j].diff(x, 2)
            K[i, j] = integrate(EI * phi_i_xx * phi_j_xx + cf * phi[i] * phi[j], (x, 0, L))
        F[i] = integrate(q0 * phi[i], (x, 0, L)) + F0 * phi[i].subs(x, L) - M0 * phi[i].diff(x).subs(x, L)
    return K, F

# Analytical solution
v_analytical = exp(beta*x)*(c1*cos(beta*x) + c2*sin(beta*x)) + exp(-beta*x)*(c3*cos(beta*x) + c4*sin(beta*x)) + q0/cf

v_prime = v_analytical.diff(x)
v_double_prime = v_analytical.diff(x, 2)
v_triple_prime = v_analytical.diff(x, 3)

# Boundary conditions
eq1 = v_analytical.subs(x, 0) - 0
eq2 = v_prime.subs(x, 0) - 0
eq3 = EI * v_double_prime.subs(x, L) + M0
eq4 = EI * v_triple_prime.subs(x, L) + F0
solution_analytical = solve([eq1, eq2, eq3.subs(L, L_val), eq4.subs(L, L_val)], [c1, c2, c3, c4])

# Numerical evaluation
x_vals = np.linspace(0, L_val, 100)
v_analytical_num = lambdify(x, v_analytical.subs(solution_analytical).subs(L, L_val), 'numpy')(x_vals)

# Compute Ritz approximations
approximations = []
for n in range(1, 6):
    #print(n)
    K, F = compute_system(n)
    c = Matrix(symbols(f'c1:{n+1}'))
    solution = solve(K * c - F, c)
    v_approx = sum(solution.get(symbols(f'c{i+1}'), 0) * phi[i] for i in range(n))
    v_approx_num = lambdify(x, v_approx.subs(L, L_val).subs(solution).evalf(), 'numpy')(x_vals)
    approximations.append((n, v_approx_num))

# Emphasize analytical solution
plt.plot(
    x_vals, v_analytical_num,
    color='black',
    linestyle='--',
    linewidth=3,
    label='Analytical Solution',
    zorder=10
)

# Plot approximations with thinner lines and color map
colors = plt.cm.viridis_r(np.linspace(0, 1, len(approximations)))
for (n, v_approx_num), color in zip(approximations, colors):
    plt.plot(
        x_vals, v_approx_num,
        label=f'Ritz Order {n} (Degree {n+1})',
        color=color,
        linewidth=1.5,
        alpha=0.9,
        zorder=5
    )

# Labels and formatting
plt.xlabel('x (m)')
plt.ylabel('Displacement v(x) (m)')
plt.title('Ritz Approximations vs Analytical Solution')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('beam_displacement_comparison.png')
plt.close()

0.6104735835807844
